<center><a href="https://sites.google.com/fat.uerj.br/livia/"> <img src="../images/capa2.png" alt="Header" style="width: 800px;"/> </a></center>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vnikoofard/DeepLearningTF/blob/main/Notebooks/05a_doggy_door.ipynb)

# Modelos pré-treinados

Embora muitas vezes seja necessário ter um conjunto de dados grande e bem anotado para resolver um desafio de aprendizado profundo, há muitos modelos pré-treinados disponíveis gratuitamente que podemos usar imediatamente. Ao decidir assumir seu próprio projeto de aprendizado profundo, é uma ótima ideia começar procurando modelos existentes on-line que possam ajudá-lo a atingir seu objetivo. Um ótimo lugar para explorar os modelos disponíveis é [NGC](https://ngc.nvidia.com/catalog/models). Também existem muitos modelos hospedados no GitHub que você pode encontrar pesquisando no Google.

## Objetivos

* Use Keras para carregar um modelo pré-treinado muito bem treinado
* Pré-processe suas próprias imagens para trabalhar com o modelo pré-treinado
* Use o modelo pré-treinado para realizar inferências precisas em suas próprias imagens

## Uma porta automática para cães

Nesta seção, criaremos uma porta para cachorros que só deixa entrar e sair cachorros (e não outros animais). Podemos manter nossos gatos dentro de casa e outros animais fora de onde eles pertencem. Usando as técnicas abordadas até agora, precisaríamos de um conjunto de dados muito grande com fotos de muitos cães, bem como de outros animais. Felizmente, existe um modelo prontamente disponível que foi treinado em um enorme conjunto de dados, incluindo muitos animais.

O [desafio ImageNet](https://en.wikipedia.org/wiki/ImageNet#History_of_the_ImageNet_challenge) produziu muitos modelos de última geração (*state-of-art*) que podem ser usados para classificação de imagens. Eles são treinados em milhões de imagens e podem classificar imagens com precisão em 1.000 categorias diferentes. Muitas dessas categorias são animais, incluindo raças de cães e gatos. Este é um modelo perfeito para nossa porta doggy.

## Carregando o Modelo

Começaremos baixando o modelo. Os modelos ImageNet treinados estão disponíveis para download diretamente na biblioteca Keras. Você pode ver os modelos disponíveis e seus detalhes [aqui](https://keras.io/api/applications/#available-models). Qualquer um desses modelos funcionaria para o nosso exercício. Escolheremos um comumente usado chamado [VGG16](https://keras.io/api/applications/vgg/):

In [ ]:
from tensorflow.keras.applications import VGG16
  
# carregar a rede VGG16 *pré-treinada* no conjunto de dados ImageNet
model = VGG16(weights="imagenet")

Agora que está carregado, vamos dar uma olhada no modelo. Parece muito com nosso modelo convolucional do exercício de língua de sinais. Preste atenção na primeira camada (a camada de entrada) e na última camada (a camada de saída). Como em nossos exercícios anteriores, precisamos garantir que nossas imagens correspondam às dimensões de entrada que o modelo espera. Também é valioso entender o que o modelo retornará da camada de saída final.

In [ ]:
model.summary()

### Dimensões de entrada
Podemos ver que o modelo espera imagens na forma de (224, 224, 3) correspondendo a 224 pixels de altura, 224 pixels de largura e 3 canais de cores. Como aprendemos em nosso último exercício, os modelos de Keras podem aceitar mais de uma imagem por vez para previsão. Se passarmos em apenas uma imagem, a forma será (1, 224, 224, 3). Precisamos garantir que, ao passar imagens para nosso modelo de previsão, elas correspondam a essas dimensões.

### Dimensões de saída
Também podemos ver que o modelo retornará uma previsão de forma 1000. Lembre-se que em nosso primeiro exercício a forma de saída de nosso modelo foi 10, correspondendo aos 10 dígitos diferentes. Em nosso segundo exercício, tivemos uma forma de 24, correspondente às 24 letras do alfabeto da língua de sinais que podem ser capturadas em uma imagem estática. Aqui, temos 1.000 categorias possíveis nas quais a imagem será colocada. Embora o conjunto de dados ImageNet completo tenha mais de 20.000 categorias, a competição e os modelos pré-treinados resultantes usam apenas um subconjunto de 1.000 dessas categorias. Podemos dar uma olhada em todas essas [categorias possíveis aqui](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).

Muitas das categorias são animais, incluindo muitos tipos de cães e gatos. Os cães são das categorias 151 a 268. Os gatos são das categorias 281 a 285. Vamos usar essas categorias para dizer ao nosso cãozinho que tipo de animal está à nossa porta e se devemos deixá-lo entrar ou não.

## Carregando uma imagem
Começaremos carregando uma imagem e exibindo-a, como fizemos nos exercícios anteriores.

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def show_image(image_path):
    image = mpimg.imread(image_path)
    print(image.shape)
    plt.imshow(image)

In [ ]:
import sys

if "google.colab" in sys.modules:
    %pip install wget
    import wget
    image_url_1 = 'https://github.com/vnikoofard/DeepLearningTF/blob/main/data/doggy_door_images/happy_dog.jpg'
    image_url_2 = 'https://github.com/vnikoofard/DeepLearningTF/blob/main/data/doggy_door_images/brown_bear.jpg'
    image_url_3 = 'https://github.com/vnikoofard/DeepLearningTF/blob/main/data/doggy_door_images/sleepy_cat.jpg'
    wget.download(image_url_1, 'brown_bear.jpg')
    wget.download(image_url_2, 'happy_dog.jpg')
    wget.download(image_url_3, 'sleepy_cat.jpg')
    show_image('happy_dog.jpg')

else:
    show_image("../data/doggy_door_images/happy_dog.jpg")

## Pré-processando a imagem

Em seguida, precisamos pré-processar a imagem para que ela esteja pronta para ser enviada ao modelo. Isso é exatamente o que fizemos em nosso último exercício, quando previmos nas imagens da língua de sinais. Lembre-se que neste caso, a forma final da imagem precisa ser (1, 224, 224, 3).

Ao carregar modelos diretamente com o Keras, também podemos aproveitar os [métodos `preprocess_input`](https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/preprocess_input). Esses métodos, associados a um modelo específico, permitem que os usuários pré-processem suas próprias imagens para corresponder às qualidades das imagens nas quais o modelo foi originalmente treinado. Tivemos que fazer isso manualmente ao realizar inferência com novas imagens ASL:

In [ ]:
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.vgg16 import preprocess_input

def load_and_process_image(image_path):
    # Imprima a forma original da imagem, para referência
    print('Original image shape: ', mpimg.imread(image_path).shape)
    
    # Carregue a imagem com um tamanho de destino de 224, 224
    image = image_utils.load_img(image_path, target_size=(224, 224))
    # Converte a imagem de um formato PIL para um array numpy
    image = image_utils.img_to_array(image)
    # Adicione uma dimensão para o número de imagens, no nosso caso 1
    image = image.reshape(1,224,224,3)
    # Pré-processar a imagem para alinhar com o conjunto de dados ImageNet original
    image = preprocess_input(image)
    # Imprimir o formato da imagem após o processamento
    print('Processed image shape: ', image.shape)
    return image

In [ ]:
if "google.colab" in sys.modules:
    processed_image = load_and_process_image("brown_bear.jpg")
else:
    processed_image = load_and_process_image("../data/doggy_door_images/brown_bear.jpg")

## Fazer uma previsão

Agora que temos nossa imagem no formato certo, podemos passá-la para nosso modelo e obter uma previsão. Esperamos uma saída de uma matriz de 1.000 elementos, que será difícil de ler. Felizmente, os modelos carregados diretamente com Keras têm outro método útil que traduzirá essa matriz de previsão em um formato mais legível.

Preencha a seguinte função para implementar a previsão:

In [ ]:
from tensorflow.keras.applications.vgg16 import decode_predictions

def readable_prediction(image_path):
    # Show image
    FIXME
    # Load and pre-process image
    image = FIXME
    # Make predictions
    predictions = FIXME
    # Print predictions in readable form
    print('Predicted:', decode_predictions(predictions, top=3))

### Solução

Clique no '...' abaixo para ver a solução.

In [ ]:
# SOLUTION
from tensorflow.keras.applications.vgg16 import decode_predictions

def readable_prediction(image_path):
    # Show image
    show_image(image_path)
    # Load and pre-process image
    image = load_and_process_image(image_path)
    # Make predictions
    predictions = model.predict(image)
    # Print predictions in readable form
    print('Predicted:', decode_predictions(predictions, top=3))

Experimente em alguns animais para ver os resultados! Também sinta-se à vontade para fazer upload de suas próprias imagens e categorizá-las apenas para ver como funciona.

In [ ]:
if "google.colab" in sys.modules:
    file_path = ""
else:
    file_path = "../data/doggy_door_images/"

In [ ]:
processed_image = readable_prediction(file_path + "brown_bear.jpg")

In [ ]:
preprocess_image = load_and_process_image(file_path + "happy_dog.jpg")

In [ ]:
processed_image = readable_prediction(file_path + "sleepy_cat.jpg")

## Só Cães

Agora que estamos fazendo previsões com nosso modelo, podemos usar nossas categorias para permitir apenas a entrada e saída de cães e manter os gatos dentro. Lembre-se de que cães são categorias 151 a 268 e gatos são categorias 281 a 285. A função [np.argmax](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) pode encontrar qual elemento da matriz de previsão é a categoria superior.

In [ ]:
import numpy as np

def doggy_door(image_path):
    show_image(image_path)
    image = load_and_process_image(image_path)
    preds = model.predict(image)
    if FIXME:
        print("Doggy come on in!")
    elif FIXME:
        print("Kitty stay inside!")
    else:
        print("You're not a dog! Stay outside!")

### Solução

Clique no '...' abaixo para ver a solução.

In [ ]:
# SOLUTION
import numpy as np

def doggy_door(image_path):
    show_image(image_path)
    image = load_and_process_image(image_path)
    preds = model.predict(image)
    if 151 <= np.argmax(preds) <= 268:
        print("Doggy come on in!")
    elif 281 <= np.argmax(preds) <= 285:
        print("Kitty stay inside!")
    else:
        print("You're not a dog! Stay outside!")

In [ ]:
if "google.colab" in sys.modules:
    file_path = ""
else:
    file_path = "../data/doggy_door_images/"

In [ ]:
doggy_door(file_path + "brown_bear.jpg")

In [ ]:
doggy_door(file_path + "happy_dog.jpg")

In [ ]:
doggy_door(file_path + "sleepy_cat.jpg")

## Resumo

Ótimo trabalho! Usando um poderoso modelo pré-treinado, criamos uma porta cachorrinha funcional em apenas algumas linhas de código. Esperamos que você esteja animado para perceber que pode aproveitar o aprendizado profundo sem muito trabalho inicial. A melhor parte é que, à medida que a comunidade de aprendizado profundo avança, mais modelos serão disponibilizados para você usar em seus próprios projetos.

### Resetar o ambiente
Antes de prosseguir, execute a célula a seguir para resetar o ambiente. Isso é necessário para passar para o próximo notebook.

In [ ]:
import os
os._exit(00)

## Next

O uso de modelos pré-treinados é incrivelmente poderoso, mas às vezes eles não são perfeitos para seus dados. Na próxima seção, você aprenderá sobre outra técnica poderosa, *aprendizado por transferência*, que permite adaptar modelos pré-treinados para fazer boas previsões para seus dados.

Continue na próxima seção: [*Modelos pré-treinados*](./05b_presidential_doggy_door.ipynb).